## Run CNMF source extraction on movies
Step 2 of the Caiman processing pipeline for dendritic two-photon calcium imaging movies. This part uses mmap files as input. These are created during motion correction with the Caiman toolbox (see `01_Preprocess_MC_3D.ipynb`). 

### Imports & Setup
The first cells import the various Python modules required by the notebook. In particular, a number of modules are imported from the Caiman package. In addition, we also setup the environment so that everything works as expected.

In [ ]:
# Generic imports
# from __future__ import absolute_import, division, print_function
# from builtins import *

import os, platform, glob, sys, re
import fnmatch
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import savemat

from IPython.display import clear_output

# Import Bokeh library
import bokeh.plotting as plotting
from bokeh.plotting import Figure, show
from bokeh.layouts import gridplot
from bokeh.models import Range1d, CrosshairTool, HoverTool, Legend
from bokeh.io import output_notebook, export_svgs
from bokeh.models.sources import ColumnDataSource

%matplotlib inline

In [ ]:
# This has to be in a separate cell, otherwise it wont work.
from bokeh import resources
output_notebook(resources=resources.INLINE)

In [ ]:
# on Linux we have to add the caiman folder to Pythonpath
if platform.system() == 'Linux':
    sys.path.append(os.path.expanduser('~/caiman'))
# environment variables for parallel processing
os.environ['MKL_NUM_THREADS']='1'
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['VECLIB_MAXIMUM_THREADS']='1'

In [ ]:
# CaImAn imports
import caiman as cm
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.components_evaluation import estimate_components_quality as estimate_q
from caiman.components_evaluation import estimate_components_quality_auto
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.source_extraction.cnmf import utilities as cnmf_utils
import caiman_utils as cm_utils

In [ ]:
from importlib import reload
reload(cm_utils)

### Select files
The following need to be specified:
- data_folder ... location of the data (relative to ~/Data)
- mc_output ... select if output of rigid ('rig') or piece-wise rigid ('els') motion correction should be used (currently only 'rig' is tested and works)
- max_files ... maximum number of files to process, e.g. for testing (if 0, all files will be processed)

In [ ]:
animal_folder = 'M3_October_2018'
date_folder = 'M3_2018-10-02'
session_folder = 'S1'
group_id = 'G0'

mc_output = 'rig'
remove_bad_frames = True # remove bad frames specified in Json file

# create the complete path to the data folder
if platform.system() == 'Linux':
    data_folder = '/home/ubuntu/Data'
elif platform.system() == 'Darwin':
    data_folder = '/Users/Henry/Data/temp/Dendrites_Gwen'
data_folder = os.path.join(data_folder, animal_folder, date_folder, session_folder)

In [ ]:
# select the mmap file created during motion correction
all_files = os.listdir(data_folder)
mmap_files = sorted([x for x in all_files if x.startswith('%s_%s' % (date_folder, session_folder)) 
           and x.endswith('.mmap') and mc_output in x and group_id in x])
n_planes = len(mmap_files)

print('Found %d mmap files. Check allocation to planes!' % (n_planes))
for i_plane in range(n_planes):
    print('Plane %d: %s' % (i_plane, mmap_files[i_plane]))

In [ ]:
mmap_files = [os.path.join(data_folder, x) for x in mmap_files]
# get metadata
for file in os.listdir(data_folder):
    if fnmatch.fnmatch(file, '%s_%s_Join_%s_*[!badFrames].json' % (date_folder, session_folder, group_id)):
        meta = json.load(open(os.path.join(data_folder,file)))
        break
trial_index = np.array(meta['trial_index'])
frame_rate = meta['frame_rate'] / n_planes

### Load data and remove bad frames

In [ ]:
fname_list = []
images_list = []
for fname in mmap_files:
    Yr, dims = cm_utils.loadData(fname)
    bad_frames, images, Y, fname_rem = cm_utils.removeBadFrames(fname, trial_index, Yr, dims, remove_bad_frames, data_folder)
    fname_list.append(fname_rem)
    images_list.append(images)

### Display frame average for each plane

In [ ]:
plt.figure(figsize=(30,60))
for ix_plane in range(n_planes):
    avg_img = np.mean(images_list[ix_plane],axis=0)
    plt.subplot(1,n_planes,ix_plane+1)
    plt.imshow(avg_img, cmap='gray'), plt.title('Frame average - Plane %d' % (ix_plane));

### Specify if plane contains dendritic signals
CaImAn uses different initialization methods depending on whether the signals are dendritic or somatic. Therefore, we need to specify the types of signal expected in each plane.

In [ ]:
is_dendritic = [True, False, True]

### Setup cluster
The default backend mode for parallel processing is through the multiprocessing package. This will allow us to use all the cores in the VM.

In [ ]:
# start the cluster (if a cluster already exists terminate it)
n_processes = 8 # number of compute processes (None to select automatically)
if 'dview' in locals() and dview is not None:
    dview.terminate()
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=n_processes, single_thread=False)

### Parameters for source extraction
Next, we define the important parameters for calcium source extraction. These parameters will have to be iteratively refined for the respective datasets.


#### Spring 2018 Parameter (dendritic)

In [ ]:
# parameters for source extraction and deconvolution
# decay_time = 0.4            # length of a typical transient in seconds
# p = 1                       # order of the autoregressive system (normally 1, 2 for fast indicators / imaging)
# gnb = 2                     # number of global background components
# merge_thresh = 0.8          # merging threshold, max correlation allowed
# rf = 10                     # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50 / None: no patches
# stride_cnmf = 5             # amount of overlap between the patches in pixels
# K = None                    # number of components per patch (usually None)
# gSig = [4, 4]               # expected half size of neurons
# method_init = 'sparse_nmf'  # initialization method (if analyzing dendritic data use 'sparse_nmf')
# is_dendrites = True         # flag for analyzing dendritic data
# #alpha_snmf = 10e2           # sparsity penalty for dendritic data analysis through sparse NMF
# alpha_snmf = 1e-6

# method_deconvolution='oasis'# deconvolution method (oasis or cvxpy)

# # parameters for component evaluation
# min_SNR = 2.5               # signal to noise ratio for accepting a component
# rval_thr = 0.8              # space correlation threshold for accepting a component
# use_cnn = False             # whether to use CNN to filter components
# cnn_thr = 0.8               # threshold for CNN based classifier

# final_rate = frame_rate             # final frame rate in Hz

#### Autumn 2018 Parameter

In [ ]:
# dataset dependent parameters
decay_time = 0.4                            # length of a typical transient in seconds

# parameters for source extraction and deconvolution
p = 1                       # order of the autoregressive system
gnb = 2                     # number of global background components
merge_thresh = 0.8          # merging threshold, max correlation allowed
rf = 25                     # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
stride_cnmf = 10             # amount of overlap between the patches in pixels
K = 20                       # number of components per patch
gSig = [4, 4]               # expected half size of neurons in pixels

method_init = 'sparse_nmf'  # initialization method (if analyzing dendritic data use 'sparse_nmf', else 'greedy_roi')
#alpha_snmf = 10e2           # sparsity penalty for dendritic data analysis through sparse NMF
alpha_snmf = 100
normalize_init = True      # default is True

ssub = 1                    # spatial subsampling during initialization
tsub = 1                    # temporal subsampling during intialization

# parameters for component evaluation
min_SNR = 2.0               # signal to noise ratio for accepting a component
rval_thr = 0.85              # space correlation threshold for accepting a component
cnn_thr = 0.99              # threshold for CNN based classifier
cnn_lowest = 0.1 # neurons with cnn probability lower than this value are rejected

In [ ]:
# create Parameters object
# unspecified parameters get default values
opts_dict = {'fnames': fname_rem,
            'fr': frame_rate,
            'decay_time': decay_time,
            'p': 1,
            'nb': gnb,
            'rf': rf,
            'K': K, 
            'stride': stride_cnmf,
            'method_init': method_init,
            'alpha_snmf': alpha_snmf,
             'normalize_init': normalize_init,
            'rolling_sum': True,
            'only_init': True,
            'ssub': ssub,
            'tsub': tsub,
            'min_SNR': min_SNR,
            'rval_thr': rval_thr,
            'use_cnn': True,
            'min_cnn_thr': cnn_thr,
            'cnn_lowest': cnn_lowest}

opts = params.CNMFParams(params_dict=opts_dict)

To get a dict with all parameters, use `opts.to_dict()`

#### Run CNMF on patches

In [51]:
opts.to_dict()

{'data': {'fnames': ['/Users/Henry/Data/temp/Dendrites_Gwen/M3_October_2018/M3_2018-10-02/S1/removedFrames_d1_109_d2_508_d3_1_order_C_frames_1094_.mmap'],
  'dims': (109, 508),
  'fr': 10.3898,
  'decay_time': 0.4,
  'dxy': (1, 1)},
 'spatial_params': {'block_size_spat': 5000,
  'dist': 3,
  'expandCore': array([[0, 0, 1, 0, 0],
         [0, 1, 1, 1, 0],
         [1, 1, 1, 1, 1],
         [0, 1, 1, 1, 0],
         [0, 0, 1, 0, 0]]),
  'extract_cc': True,
  'maxthr': 0.1,
  'medw': (3, 3),
  'method_exp': 'dilate',
  'method_ls': 'lasso_lars',
  'n_pixels_per_process': 55372,
  'nb': 2,
  'normalize_yyt_one': True,
  'nrgthr': 0.9999,
  'num_blocks_per_run_spat': 20,
  'se': array([[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]], dtype=uint8),
  'ss': array([[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]], dtype=uint8),
  'thr_method': 'nrg',
  'update_background_components': True},
 'temporal_params': {'ITER': 2,
  'bas_nonneg': False,
  'block_size_temp': 5000,
  'fudge_factor':

In [54]:
opts.get_group('init')

{'K': 20,
 'alpha_snmf': 100,
 'center_psf': False,
 'gSig': [5, 5],
 'gSiz': [11, 11],
 'init_iter': 2,
 'kernel': None,
 'maxIter': 5,
 'max_iter_snmf': 500,
 'method_init': 'sparse_nmf',
 'min_corr': 0.85,
 'min_pnr': 20,
 'nIter': 5,
 'nb': 2,
 'normalize_init': True,
 'options_local_NMF': None,
 'perc_baseline_snmf': 20,
 'ring_size_factor': 1.5,
 'rolling_length': 100,
 'rolling_sum': True,
 'sigma_smooth_snmf': (0.5, 0.5, 0.5),
 'ssub': 1,
 'ssub_B': 2,
 'tsub': 1}

In [ ]:
# First extract spatial and temporal components on patches and combine them
# for this step deconvolution is turned off (p=0)
# Then re-run seeded CNMF on accepted patches to refine and perform deconvolution
opts.set('temporal', {'p': 0})
cnm_list = []
for ix_plane in range(n_planes):
    if is_dendritic[ix_plane]:
        opts.set('init', {'method_init': 'sparse_nmf'})
    else:
        opts.set('init', {'method_init': 'greedy_roi'})
    cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
    cnm.fit(images_list[ix_plane])
     
    cnm.params.set('temporal', {'p': p})
    cnm2 = cnm.refit(images_list[ix_plane], dview=dview)
    
    cnm_list.append(cnm2)
    
    clear_output()

In [60]:
# Evaluate components
for ix_plane, cnm in enumerate(cnm_list):
    cnm.estimates.evaluate_components(images_list[ix_plane], cnm.params, dview=dview)
    cnm_list[ix_plane] = cnm
    print('\nPlane %d' % (ix_plane))
    print('Found %d good / %d bad components\n' % (len(cnm.estimates.idx_components), 
                                                 len(cnm.estimates.idx_components_bad)))

USING MODEL:/Users/Henry/caiman_data/model/cnn_model.json
20/20 [==============================] - 0s 8ms/step

Plane 0
Found 7 good / 13 bad components

USING MODEL:/Users/Henry/caiman_data/model/cnn_model.json
13/13 [==============================] - 0s 12ms/step

Plane 1
Found 13 good / 0 bad components

USING MODEL:/Users/Henry/caiman_data/model/cnn_model.json
20/20 [==============================] - 0s 9ms/step

Plane 2
Found 2 good / 18 bad components



Plot contours of selected and rejected components

In [ ]:
cnm2.estimates.plot_contours_nb(img=Cn, idx=cnm2.estimates.idx_components)

View traces of accepted and rejected components. Note that if you get data rate error you can start Jupyter notebooks using: 'jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10'

In [ ]:
# accepted components
cnm2.estimates.nb_view_components(img=Cn, idx=cnm2.estimates.idx_components)

In [ ]:
# rejected components
if len(cnm2.estimates.idx_components_bad) > 0:
    cnm2.estimates.nb_view_components(img=Cn, idx=cnm2.estimates.idx_components_bad)
else:
    print("No components were rejected.")

#### Extract DF/F values

In [ ]:
cnm2.estimates.detrend_df_f(quantileMin=8, frames_window=250)

Select only high quality components.

In [ ]:
cnm2.estimates.select_components(use_object=True)

In [ ]:
cnm2.estimates.nb_view_components(img=Cn, denoised_color='red')